# Antes de usar esse código

Faça download do aqruivo .csv de todas as suas transações na plataforma da NovaDAX durante o ano de 2020

# O que eu preciso para declarar o IR 2020

Ficha de Bens e Direitos
- 1) Minhas posições em 31/12/19: o quanto eu tinha de cada crypto
- 2) Minhas posições em 31/12/20 e qual foi o preço médio de aquisição de cada crypto -> pra declarar o preço de custo em bens e direitos

Ficha de lucros tributáveis
- 3) Se eu vendi mais que 35k em um mês -> teria que ter pago darf sobre o lucro
- 4) Identificar o mês e o valor do lucro naquele mês

Ficha de lucros não tributáveis
- Acredito que não seja necessário

In [209]:
## Importando os dados

import pandas as pd

transacoes = pd.read_csv('Extrato de ativos.csv') # O aquivo precisa estar na mesma pasta desse código ou o caminho completo
transacoes.head()

,(UTC)HistoryTradesWsData,Tipo,Moeda,Valor,Saldo após,Status
0,30/12/2020 20:16:37,Compra(ETH/BTC),BTC,"-0,0009975BTC(≈R$150.06)","+0,00540457",Sucesso
1,30/12/2020 20:16:37,Taxa de transação,ETH,"-0,0001905ETH(≈R$0.75)","+1,16375829",Sucesso
2,30/12/2020 20:16:37,Compra(ETH/BTC),ETH,"+0,0381ETH(≈R$149.79)","+1,16394879",Sucesso
3,27/12/2020 11:24:50,Venda(DGB/USDT),DGB,"-500,00DGB(≈R$84.41)","+997,069",Sucesso
4,27/12/2020 11:24:50,Taxa de transação,USDT,"-0,0477USDT(≈R$0.25)","+15,859461",Sucesso


In [118]:
## Lista de moedas que eu negociei
todas_moedas = transacoes['Moeda'].unique()
todas_moedas

array(['BTC', 'ETH', 'DGB', 'USDT', 'BRL', 'LTC', 'LINK', 'XMR', 'OMG',
       'ADA', 'BTT', 'BNB', 'XRP', 'EOS', 'BSV', 'TRX', 'DASH', 'WAVES',
       'IOTA', 'XLM'], dtype=object)

# Passo 1) Posição no fim de 2019

Confirmar se todas as moedas foram adquiridas em 2020 ou antes (se forem antes o resto do código não funciona)

Assim, o saldo anterior da ficha de bens e direitos é R$0,00

In [55]:
## Função auxiliar para converter valores em numero

def RobustStringFormating(s,moeda):
    
    s_1 = s.split('(')[0].replace(moeda,'').replace('R$','')
    s_2 = s_1.replace('.',',').split(',')
    s_decimais = s_2[-1]
    
    s_unidades = ''
    for part in s_2[:-1]:
        s_unidades += part

    return float(s_unidades+'.'+s_decimais)

In [176]:
## Passo 1.1. Posição no fim de 2019: 
# confirmar se todas as moedas foram adquiridas em 2020 ou antes (se for antes não funciona) -> Saldo anterior 0 reais
##

## Jeito mais simples: encontrar a transações onde o saldo da moeda é igual ao valor comprado 
# isso sinaliza que o saldo em no fim de 2019 era 0

index_primeira_compra = []

for moeda in todas_moedas:
    transacoes_moeda = transacoes[transacoes['Moeda']==moeda]

    # Buscando a primeira transação de 2020 de cada moeda
    for idx,row in transacoes_moeda.iloc[::-1].iterrows():

        quantidade_transacao = RobustStringFormating(row['Valor'],moeda)
        saldo = RobustStringFormating(row['Valor'],moeda)

        # Se a primeira transação for identificada nesse dataset, isso significa que o saldo era 0 em 2019
        if quantidade_transacao == saldo:
            index_primeira_compra.append(idx)
        
        break
        
if len(todas_moedas) == len(index_primeira_compra):
    print('Todas as moedas transacionadas foram compradas em 2020') # Subs pelo ano do dataset
    print('Isso significa que o "Saldo anterior" a ser preenchido em "Bens e direitos" é zero para todas as moedas')
    
else: print('Alguma moeda foi transacionada em anos anteriores, verifique seus dados')

Todas as moedas transacionadas foram compradas em 2020
Isso significa que o "Saldo anterior" a ser preenchido em "Bens e direitos" é zero para todas as moedas


# Passo 2) Posição no fim de 2020: saldo da ultima transação do ano e custo totoal de aquisição

Primeiro, fiz algumas funções auxiliares, que poderia ter aplicado ao dataset inteiro logo de cada -> TO DO

Depois temos a função que calcula o preço médio de aquisição, sua posição no fim de 2020 e o custo total de compra

In [60]:
## Uma função pra converter só os valor que estão em formato de string indicando BRL

def converterStrigReaisEmFloat(string_reais):
    return float(string_reais.replace('R$','').replace(',','.'))

In [188]:
## Função para pegar só a quantidade de moeda
# Eu precisei fazer essa função com ifs e não com replace/split pq eu n queri precisar de pedir como argumento da moeda

import numpy as np

def quantidadeDeMoedaParaFloat(string):
    novo_string = ''

    for chrt in string:

        # Condição de parada
        if chrt in str(np.arange(0,10)) or chrt in ['+','-','.',',']:

            # Substituição
            if chrt == ',':
                novo_chrt = '.'

            elif chrt == '.':
                novo_chrt = ''

            else: novo_chrt = chrt

            novo_string += novo_chrt

        else: break
            
    # Correção para um bug nos dados (valores com duas virgulas)
    if novo_string.count('.') > 1:
        i = novo_string[::-1].find('.')
        novo_string = novo_string[:-i-1].replace('.','')+novo_string[-i-1:]

    return float(novo_string)

# Teste da função
moeda = 'DGB'
base = 'USDT'
saldo_da_moeda = transacoes[transacoes['Tipo'] == 'Compra('+moeda+'/'+base+')'][::2]
string = saldo_da_moeda['Valor'].iloc[1]
quantidadeDeMoedaParaFloat(string)

-49.997024

In [136]:
## Transforma uma string do formato -10,509346USDT(≈R$54.39) em um float 54.39

def valorParaFloat(string):
    nova_string = ''
    gravar = False
    
    for chrt in string[:-1]:
        
        if gravar:
            nova_string += chrt

        if chrt == '$':
            gravar = True

    return float(nova_string)

# Teste da função
string = saldo_da_moeda['Valor'].iloc[0]
print(string)
valorEmBRLParaFloat(string)

-23,95USDT(≈R$135.21)


135.21

In [208]:
## 
# Passo 1.2. Posição no fim de 2020: saldo da ultima transação do ano
##

def CustoMedio(moeda,transacoes):
    
    ## Ultima posição de 2020
    transacoes_moeda = transacoes[transacoes['Moeda']==moeda]
    ultima_posicao = RobustStringFormating(transacoes_moeda['Saldo após'].iloc[0],moeda)

    ## Descobrindo o preço médio

    # As compras podem ter sido feitas em: BRL, USDT, BTC e ETH
    bases = ['BRL','USDT','BTC','ETH']
    valores_medios_parciais = []

    for base in bases:

        if base == moeda: pass

        else:

            saldo_da_moeda = transacoes[transacoes['Tipo'] == 'Compra('+moeda+'/'+base+')'][::2]
            saldo_da_base = transacoes[transacoes['Tipo'] == 'Compra('+moeda+'/'+base+')'][1::2]
            # As duas precisam ter o mesmo numero de linhas ----> colocar um if pra conferir aqui

            # Se houver transações daquele par
            if len(saldo_da_moeda) != 0:
                # Se a base for BRL, o preço médio é diretamente calculado com os dados do dataset
                if base == 'BRL':

                    saldo_da_base_float = saldo_da_base['Valor'].apply(converterStrigReaisEmFloat)
                    saldo_da_moeda_float = saldo_da_moeda['Valor'].apply(quantidadeDeMoedaParaFloat)

                    valor_medio_parcial = -np.mean(saldo_da_base_float.values/saldo_da_moeda_float.values)

                else:
                    saldo_da_base_float = saldo_da_base['Valor'].apply(valorParaFloat)
                    saldo_da_moeda_float = saldo_da_base['Valor'].apply(quantidadeDeMoedaParaFloat)

                    valor_medio_parcial = np.mean(saldo_da_base_float.values/saldo_da_moeda_float.values)


                valores_medios_parciais.append(valor_medio_parcial)
                
    
    # Preço médio
    preco_medio = np.array(valores_medios_parciais).mean()
    print('O seu valor médio é R$'+str(preco_medio),'em uma posição final de',str(ultima_posicao)+moeda)
    
    # Custo final total
    custo_medio_moeda = preco_medio*ultima_posicao
    print('O seu custo de aquisição final de '+moeda+' é: R$'+str(custo_medio_moeda))
    
    return custo_medio_moeda

## Teste da função 
moeda = 'OMG'
CustoMedio(moeda,transacoes)

O seu valor médio é R$24.404101455397978 em uma posição final de 3.9e-05OMG
O seu custo de aquisição final de OMG é: R$0.0009517599567605211


0.0009517599567605211

In [205]:
## Aplicando para todas as moedas

for moeda in todas_moedas:
    
    if moeda =='BRL': pass
    
    else:
        print(moeda)
        CustoMedio(moeda,transacoes)

BTC
O seu valor médio é R$61569.51054153113 em uma posição final de 0.00540457BTC
O seu custo de aquisição final de BTC é: R$332.75672958744286
ETH
O seu valor médio é R$2702.7541313983675 em uma posição final de 1.16375829ETH
O seu custo de aquisição final de ETH é: R$3145.3525262465996
DGB
O seu valor médio é R$0.15605538456688944 em uma posição final de 997.069DGB
O seu custo de aquisição final de DGB é: R$155.5979862347239
USDT
O seu valor médio é R$5.441445784055773 em uma posição final de 15.859461USDT
O seu custo de aquisição final de USDT é: R$86.29839719584695
LTC
O seu valor médio é R$399.2546658862094 em uma posição final de 1.49311951LTC
O seu custo de aquisição final de LTC é: R$596.1349310932308
LINK
O seu valor médio é R$68.86108807308253 em uma posição final de 1.00465LINK
O seu custo de aquisição final de LINK é: R$69.18129213262236
XMR
O seu valor médio é R$535.0514600398881 em uma posição final de 0.4084365XMR
O seu custo de aquisição final de XMR é: R$218.5345456585

# 3) Vendas mensais